In [ ]:

from google.cloud import bigquery
from google.oauth2 import service_account
from zipfile import ZipFile
import os
from time import sleep

In [ ]:
# Connect to GBQ. 
service_path = 'C:\\Users\\markmcdiffett\\Desktop\\2021fallsemester\\Applieddataanalytics\\'
service_file = 'wedgeproject-327716-5f983081c235.json' # change this to your authentication information  
gbq_proj_id = 'wedgeproject-327716' # change this to your poroject. 
gbq_dataset_id = 'wedge_users'

# And this should stay the same. 
private_key =service_path + service_file

In [ ]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
#get list of files 
file_list=os.listdir("C:\\Users\\markmcdiffett\\Desktop\\2021fallsemester\\Applieddataanalytics\\Wedge\\WedgeFiles_Clean\\clean-files")


In [ ]:
#remove ".csv" and create list to insert into load to GBQ function
full_file_list = []
for file in file_list:
    full_file_list.append(file.replace(".csv",""))
print(full_file_list)

In [ ]:
#schema of table
schema = [
        bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
]

In [ ]:
#function takes file name and load all data to GBQ instance.
def load_to_gbq(file_name):
    my_table = file_name
    table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])
    table_ref = client.create_table(table = table_full_name)
    table = client.get_table(table_ref)
    print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))
    
    #job config info for transforming files to GBQ
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    job_config.schema_update_options = [
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION # This allows us to modify the table. 
    ]
    job_config.schema = schema
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.skip_leading_rows = 1
    
    #opens file and converts to GBQ
    with open("C:\\Users\\markmcdiffett\\Desktop\\2021fallsemester\\Applieddataanalytics\\Wedge\\WedgeFiles_Clean\\clean-files\\{}.csv".format(my_table), "rb") as source_file:
        job = client.load_table_from_file(
            source_file,
            table_ref,
            location="US",  # Must match the destination dataset location.
            job_config=job_config,
        )  # API request
    
    print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))

In [ ]:
files_list = ['transArchive_201001_201003_clean', 'transArchive_201004_201006_clean', 'transArchive_201007_201009_clean', 'transArchive_201010_201012_clean', 'transArchive_201101_201103_clean', 'transArchive_201104_clean', 'transArchive_201105_clean', 'transArchive_201106_clean', 'transArchive_201107_201109_clean', 'transArchive_201110_201112_clean', 'transArchive_201201_201203_clean', 'transArchive_201201_201203_inactive_clean', 'transArchive_201204_201206_clean', 'transArchive_201204_201206_inactive_clean', 'transArchive_201207_201209_clean', 'transArchive_201207_201209_inactive_clean', 'transArchive_201210_201212_clean', 'transArchive_201210_201212_inactive_clean', 'transArchive_201301_201303_clean', 'transArchive_201301_201303_inactive_clean', 'transArchive_201304_201306_clean', 'transArchive_201304_201306_inactive_clean', 'transArchive_201307_201309_clean', 'transArchive_201307_201309_inactive_clean', 'transArchive_201310_201312_clean', 'transArchive_201310_201312_inactive_clean', 'transArchive_201401_201403_clean', 'transArchive_201401_201403_inactive_clean', 'transArchive_201404_201406_clean', 'transArchive_201404_201406_inactive_clean', 'transArchive_201407_201409_clean', 'transArchive_201407_201409_inactive_clean', 'transArchive_201410_201412_clean', 'transArchive_201410_201412_inactive_clean', 'transArchive_201501_201503_clean', 'transArchive_201504_201506_clean', 'transArchive_201507_201509_clean', 'transArchive_201510_clean', 'transArchive_201511_clean', 'transArchive_201512_clean', 'transArchive_201601_clean', 'transArchive_201602_clean', 'transArchive_201603_clean', 'transArchive_201604_clean', 'transArchive_201605_clean', 'transArchive_201606_clean', 'transArchive_201607_clean', 'transArchive_201608_clean', 'transArchive_201609_clean', 'transArchive_201610_clean', 'transArchive_201611_clean', 'transArchive_201612_clean', 'transArchive_201701_clean']

In [ ]:
for item in files_list:
    load_to_gbq(item)